Stage 2 - Process and clean the raw data and save the processed data in log files

#### Importing all necessary libraries

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from collections import Counter

import json
import sys
from nltk.corpus import stopwords, sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Reading the previous json file

In [2]:
data = pd.read_json('data.json')

In [3]:
data.head()           #top 5 tweets

,created_at,id_str,text,user,place,geo,coordinates
0,2021-12-15 04:38:51+00:00,1470976626074734592,"RT @IDontrustBiden: Los Angeles schools, with ...","{'id': 2759339413, 'id_str': '2759339413', 'na...",NaN,NaN,NaN
1,2021-12-15 04:38:51+00:00,1470976626045431808,RT @HermelaTV: I see parallels in ppl of Globa...,"{'id': 1356814714123206658, 'id_str': '1356814...",NaN,NaN,NaN
2,2021-12-15 04:38:51+00:00,1470976625957240832,RT @DrakeGatsby: YOUR MOM IS SO OLD she’s at a...,"{'id': 142026927, 'id_str': '142026927', 'name...",NaN,NaN,NaN
3,2021-12-15 04:38:51+00:00,1470976625235869696,RT @RMConservative: 4th Child Died in Vietnam ...,"{'id': 57808228, 'id_str': '57808228', 'name':...",NaN,NaN,NaN
4,2021-12-15 04:38:51+00:00,1470976624795467776,RT @CP24: Feds could announce tougher COVID-19...,"{'id': 879456547612614658, 'id_str': '87945654...",NaN,NaN,NaN


In [4]:
df=data[['text']] #For the cleaning operation we only need text coloumn

In [5]:
df.head()

,text
0,"RT @IDontrustBiden: Los Angeles schools, with ..."
1,RT @HermelaTV: I see parallels in ppl of Globa...
2,RT @DrakeGatsby: YOUR MOM IS SO OLD she’s at a...
3,RT @RMConservative: 4th Child Died in Vietnam ...
4,RT @CP24: Feds could announce tougher COVID-19...


### Cleaning the text

#### We will create a function to clean the tweets

In [6]:
def cleantxt(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) #Removing @ mentions
    text = re.sub(r'#', '', text) #removing the "#" symbols
    text = re.sub(r'RT[\s]+', '', text) #removing RT
    text = re.sub(r'https?:\/\/\s+', '',text) #removing the hyper link
    text = re.sub(r'(www\.[^\s]+)|(https?://[^\s]+)','',text) # removing emails
        
    return text

#### Cleaning tweets

In [7]:
df['text']= df['text'].apply(cleantxt)

#### Displaying Cleaned data

In [8]:
df

,text
0,": Los Angeles schools, with 30K unvaccinated s..."
1,: I see parallels in ppl of Global South sayin...
2,: YOUR MOM IS SO OLD she’s at an elevated risk...
3,: 4th Child Died in Vietnam Due to ‘Overreacti...
4,: Feds could announce tougher COVID-19 border ...
5,_miumiu: VP for University Relations Joel Mali...
6,": Maybe it’s just me, but after 800,000 U.S. C..."
7,": ""I see a lot of vaccinated people getting CO..."
8,: CDC Confirms 80% of COVID-19 Cases Caused by...
9,: UPDATE: PNP Health Service recorded the foll...


In [9]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

#### Displaying text after removing all emojis

In [10]:
df['text']= df['text'].apply(remove_emoji)

df

,text
0,": Los Angeles schools, with 30K unvaccinated s..."
1,: I see parallels in ppl of Global South sayin...
2,: YOUR MOM IS SO OLD she’s at an elevated risk...
3,: 4th Child Died in Vietnam Due to ‘Overreacti...
4,: Feds could announce tougher COVID-19 border ...
5,_miumiu: VP for University Relations Joel Mali...
6,": Maybe it’s just me, but after 800,000 U.S. C..."
7,": ""I see a lot of vaccinated people getting CO..."
8,: CDC Confirms 80% of COVID-19 Cases Caused by...
9,: UPDATE: PNP Health Service recorded the foll...


In [11]:
def tranform_text(text):
    
    text = text.lower()
    text = nltk.word_tokenize(text) 
    #text = re.sub(r'RT[\s]+', '', text) #removing RT
    
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
            
            
    text = y[:]
    y.clear()
    
    custom_list = ['rt']
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
    
    return " ".join(y)

df['transformed_text'] = df['text'].apply(tranform_text)

df

,text,transformed_text
0,": Los Angeles schools, with 30K unvaccinated s...",los angeles schools 30k unvaccinated students ...
1,: I see parallels in ppl of Global South sayin...,see parallels ppl global south saying nomore w...
2,: YOUR MOM IS SO OLD she’s at an elevated risk...,mom old elevated risk covid hope taking extra ...
3,: 4th Child Died in Vietnam Due to ‘Overreacti...,4th child died vietnam due overreaction vaccine
4,: Feds could announce tougher COVID-19 border ...,feds could announce tougher border measures we...
5,_miumiu: VP for University Relations Joel Mali...,vp university relations joel malina virtually ...
6,": Maybe it’s just me, but after 800,000 U.S. C...",maybe covid deaths think time give person year...
7,": ""I see a lot of vaccinated people getting CO...",see lot vaccinated people getting covid point ...
8,: CDC Confirms 80% of COVID-19 Cases Caused by...,cdc confirms 80 cases caused omicron variant u...
9,: UPDATE: PNP Health Service recorded the foll...,update pnp health service recorded following t...


#### Tokenise string

In [12]:
df['tokenise'] = df.apply(lambda row: nltk.word_tokenize(row[1]), axis=1)

df

,text,transformed_text,tokenise
0,": Los Angeles schools, with 30K unvaccinated s...",los angeles schools 30k unvaccinated students ...,"[los, angeles, schools, 30k, unvaccinated, stu..."
1,: I see parallels in ppl of Global South sayin...,see parallels ppl global south saying nomore w...,"[see, parallels, ppl, global, south, saying, n..."
2,: YOUR MOM IS SO OLD she’s at an elevated risk...,mom old elevated risk covid hope taking extra ...,"[mom, old, elevated, risk, covid, hope, taking..."
3,: 4th Child Died in Vietnam Due to ‘Overreacti...,4th child died vietnam due overreaction vaccine,"[4th, child, died, vietnam, due, overreaction,..."
4,: Feds could announce tougher COVID-19 border ...,feds could announce tougher border measures we...,"[feds, could, announce, tougher, border, measu..."
5,_miumiu: VP for University Relations Joel Mali...,vp university relations joel malina virtually ...,"[vp, university, relations, joel, malina, virt..."
6,": Maybe it’s just me, but after 800,000 U.S. C...",maybe covid deaths think time give person year...,"[maybe, covid, deaths, think, time, give, pers..."
7,": ""I see a lot of vaccinated people getting CO...",see lot vaccinated people getting covid point ...,"[see, lot, vaccinated, people, getting, covid,..."
8,: CDC Confirms 80% of COVID-19 Cases Caused by...,cdc confirms 80 cases caused omicron variant u...,"[cdc, confirms, 80, cases, caused, omicron, va..."
9,: UPDATE: PNP Health Service recorded the foll...,update pnp health service recorded following t...,"[update, pnp, health, service, recorded, follo..."


In [13]:
df = df['tokenise']

In [14]:
df

0     [los, angeles, schools, 30k, unvaccinated, stu...
1     [see, parallels, ppl, global, south, saying, n...
2     [mom, old, elevated, risk, covid, hope, taking...
3     [4th, child, died, vietnam, due, overreaction,...
4     [feds, could, announce, tougher, border, measu...
5     [vp, university, relations, joel, malina, virt...
6     [maybe, covid, deaths, think, time, give, pers...
7     [see, lot, vaccinated, people, getting, covid,...
8     [cdc, confirms, 80, cases, caused, omicron, va...
9     [update, pnp, health, service, recorded, follo...
10    [pandemic, 75, lawmakers, traded, stock, compa...
11                  [wear, mask, easiest, thing, covid]
12    [many, families, lost, earnings, working, moms...
13    [broadly, ominous, whole, one, sense, covid, t...
14    [uncle, took, dinner, tonight, belated, bday, ...
15    [want, learn, workplace, experiences, particip...
16                  [thought, weather, catching, covid]
17    [covid, bottlenecks, hold, back, german, e

#### Creating a CSV file for saving tweets

In [17]:
df.to_csv("tweets.csv")